In [16]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle 
from fastparquet import write, ParquetFile

In [4]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR
import pickle 
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [5]:
df=pd.read_csv("Airbnb_Open_Data.csv")
df.head()

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (25) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,id,NAME,host id,host_identity_verified,host name,neighbourhood group,neighbourhood,lat,long,country,...,service fee,minimum nights,number of reviews,last review,reviews per month,review rate number,calculated host listings count,availability 365,house_rules,license
0,1001254,Clean & quiet apt home by the park,80014485718,unconfirmed,Madaline,Brooklyn,Kensington,40.64749,-73.97237,United States,...,$193,10.0,9.0,10/19/2021,0.21,4.0,6.0,286.0,Clean up and treat the home the way you'd like...,NaN
1,1002102,Skylit Midtown Castle,52335172823,verified,Jenna,Manhattan,Midtown,40.75362,-73.98377,United States,...,$28,30.0,45.0,5/21/2022,0.38,4.0,2.0,228.0,Pet friendly but please confirm with me if the...,NaN
2,1002403,THE VILLAGE OF HARLEM....NEW YORK !,78829239556,NaN,Elise,Manhattan,Harlem,40.80902,-73.94190,United States,...,$124,3.0,0.0,NaN,NaN,5.0,1.0,352.0,"I encourage you to use my kitchen, cooking and...",NaN
3,1002755,NaN,85098326012,unconfirmed,Garry,Brooklyn,Clinton Hill,40.68514,-73.95976,United States,...,$74,30.0,270.0,7/5/2019,4.64,4.0,1.0,322.0,NaN,NaN
4,1003689,Entire Apt: Spacious Studio/Loft by central park,92037596077,verified,Lyndon,Manhattan,East Harlem,40.79851,-73.94399,United States,...,$41,10.0,9.0,11/19/2018,0.10,3.0,1.0,289.0,"Please no smoking in the house, porch or on th...",NaN


In [6]:
#I created a copy of the data set
df1=df.copy()

In [8]:
#I shuffled the data set
df1=df1.sample(frac = 1)
df1.head()

,id,NAME,host id,host_identity_verified,host name,neighbourhood group,neighbourhood,lat,long,country,...,service fee,minimum nights,number of reviews,last review,reviews per month,review rate number,calculated host listings count,availability 365,house_rules,license
98077,55169263,Sun-Drenched Perch in Charming Brownstone,63243790047,verified,Joyce,Brooklyn,Carroll Gardens,40.68112,-73.99296,United States,...,$179,4.0,9.0,5/18/2019,0.87,2.0,1.0,0.0,We expect that you would treat the house as if...,NaN
58498,33309781,★NEW★ | ❤eART of Brklyn | Queen Suite + Free pkng,31783882047,verified,Ashley,Brooklyn,Cypress Hills,40.67953,-73.89488,United States,...,$39,2.0,102.0,1/3/2022,3.67,5.0,7.0,270.0,NaN,NaN
4334,3395004,Carroll Gardens Brooklyn Apartment,28441288804,unconfirmed,Derek,Brooklyn,Carroll Gardens,40.67786,-73.99871,United States,...,$60,6.0,2.0,8/7/2015,0.04,1.0,1.0,299.0,"Please no smoking, pets or candles in the apar...",NaN
65968,37435462,"Sunny, modern Clinton Hill Apartment",2068477009,unconfirmed,David,Brooklyn,Bedford-Stuyvesant,40.68699,-73.95617,United States,...,$134,30.0,4.0,2/28/2021,0.07,2.0,1.0,0.0,NaN,NaN
5839,4226216,Beautiful 1 Bedroom in Midtown West,18679024795,unconfirmed,Lev,Manhattan,Hell's Kitchen,40.76443,-73.99184,United States,...,$13,3.0,2.0,1/3/2015,0.03,4.0,1.0,400.0,No smoking No pets We don't charge a cleaning ...,NaN


In [9]:
#I seperated the needed columns from the dataset
cat_col=['host_identity_verified','neighbourhood group',
        'instant_bookable', 'cancellation_policy', 'room type']
num_col=['Construction year', 'service fee', 'minimum nights',
       'number of reviews']

target=['price']
    
    

In [10]:
df1=df1[cat_col+num_col+target]

In [11]:
df1.reset_index(drop=True,inplace=True)
df1.head()

,host_identity_verified,neighbourhood group,instant_bookable,cancellation_policy,room type,Construction year,service fee,minimum nights,number of reviews,price
0,verified,Brooklyn,False,strict,Entire home/apt,2016.0,$179,4.0,9.0,$897
1,verified,Brooklyn,False,strict,Entire home/apt,2022.0,$39,2.0,102.0,$196
2,unconfirmed,Brooklyn,True,flexible,Entire home/apt,2005.0,$60,6.0,2.0,$301
3,unconfirmed,Brooklyn,True,moderate,Entire home/apt,2015.0,$134,30.0,4.0,$672
4,unconfirmed,Manhattan,False,strict,Entire home/apt,2020.0,$13,3.0,2.0,$66


In [12]:
df1.dropna(inplace=True)

In [13]:
#I split the dataset into my train, test and validation sets
train, val, test = \
              np.split(df1.sample(frac=1, random_state=42), 
                       [int(.6*len(df1)), int(.8*len(df1))])

In [19]:
#I converted the test set to a parquet file for ease of storage
test.to_parquet("./test_par.parquet", compression="GZIP")


In [12]:
train.reset_index(drop=True,inplace=True) 
val.reset_index(drop=True,inplace=True)
test.reset_index(drop=True,inplace=True) 

In [13]:
train.to_csv("train.csv")
val.to_csv("val.csv")
test.to_csv("test.csv")

In [14]:
train.head()

,host_identity_verified,neighbourhood group,instant_bookable,cancellation_policy,room type,Construction year,service fee,minimum nights,number of reviews,price
0,unconfirmed,Brooklyn,False,flexible,Entire home/apt,2019.0,$26,1.0,4.0,$131
1,verified,Brooklyn,False,flexible,Private room,2012.0,$73,3.0,47.0,$367
2,verified,Manhattan,False,moderate,Entire home/apt,2021.0,$140,30.0,59.0,$699
3,verified,Brooklyn,False,strict,Entire home/apt,2008.0,$145,7.0,41.0,$725
4,unconfirmed,Manhattan,False,flexible,Entire home/apt,2016.0,$227,1.0,56.0,"$1,135"


In [15]:
fees_col=['price','service fee']
for col in fees_col:
    train[col]=train[col].apply (lambda x:int(x.split("$")[1].rstrip().replace(",","")))

In [16]:
for col in train[num_col]:
    train[col]=train[col].astype(int)
    

In [17]:
train

,host_identity_verified,neighbourhood group,instant_bookable,cancellation_policy,room type,Construction year,service fee,minimum nights,number of reviews,price
0,unconfirmed,Brooklyn,False,flexible,Entire home/apt,2019,26,1,4,131
1,verified,Brooklyn,False,flexible,Private room,2012,73,3,47,367
2,verified,Manhattan,False,moderate,Entire home/apt,2021,140,30,59,699
3,verified,Brooklyn,False,strict,Entire home/apt,2008,145,7,41,725
4,unconfirmed,Manhattan,False,flexible,Entire home/apt,2016,227,1,56,1135
...,...,...,...,...,...,...,...,...,...,...
60580,verified,Manhattan,True,strict,Entire home/apt,2015,179,2,6,895
60581,verified,Manhattan,True,moderate,Entire home/apt,2019,26,1,15,128
60582,verified,Brooklyn,True,moderate,Private room,2010,191,4,1,956
60583,unconfirmed,Manhattan,False,moderate,Entire home/apt,2017,198,365,27,988


In [18]:
X=train.drop('price',axis=1)

train_dicts = X.to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)


y_train = train['price'].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

r2_score(y_train, y_pred)

0.9999818119619818

In [19]:
with open('./lr.bin', 'wb') as f_out:
            pickle.dump((dv,lr), f_out)

In [41]:
#I built a preprocessin pipeline
def preprocess(filename):
    df = pd.read_csv(filename)
    cat_col=['host_identity_verified','neighbourhood group',
        'instant_bookable', 'cancellation_policy', 'room type']
    num_col=['Construction year', 'service fee', 'minimum nights',
       'number of reviews']
    target=['price']
    
    df=df[cat_col+num_col+target]
    
    fees_col=['price','service fee']
    for col in fees_col:
        df[col]=df[col].apply (lambda x:int(x.split("$")[1].rstrip().replace(",","")))
        
    for col in df[num_col]:
        df[col]=df[col].astype(int)
    
    return df 
    

In [20]:
val 

,host_identity_verified,neighbourhood group,instant_bookable,cancellation_policy,room type,Construction year,service fee,minimum nights,number of reviews,price
0,unconfirmed,Manhattan,True,moderate,Private room,2015.0,$31,1.0,0.0,$156
1,verified,Brooklyn,True,strict,Entire home/apt,2005.0,$32,30.0,7.0,$161
2,verified,Manhattan,True,strict,Private room,2019.0,$164,1.0,3.0,$818
3,verified,Brooklyn,False,moderate,Entire home/apt,2015.0,$238,30.0,3.0,"$1,188"
4,verified,Brooklyn,True,moderate,Entire home/apt,2009.0,$73,3.0,43.0,$364
...,...,...,...,...,...,...,...,...,...,...
20190,unconfirmed,Bronx,False,moderate,Shared room,2007.0,$140,2.0,2.0,$702
20191,unconfirmed,Brooklyn,False,flexible,Private room,2008.0,$159,5.0,0.0,$794
20192,unconfirmed,Manhattan,False,flexible,Entire home/apt,2013.0,$143,11.0,14.0,$715
20193,verified,Brooklyn,True,flexible,Private room,2019.0,$22,5.0,1.0,$111


In [21]:
val1=preprocess("./val.csv") 
val1

,host_identity_verified,neighbourhood group,instant_bookable,cancellation_policy,room type,Construction year,service fee,minimum nights,number of reviews,price
0,unconfirmed,Manhattan,True,moderate,Private room,2015,31,1,0,156
1,verified,Brooklyn,True,strict,Entire home/apt,2005,32,30,7,161
2,verified,Manhattan,True,strict,Private room,2019,164,1,3,818
3,verified,Brooklyn,False,moderate,Entire home/apt,2015,238,30,3,1188
4,verified,Brooklyn,True,moderate,Entire home/apt,2009,73,3,43,364
...,...,...,...,...,...,...,...,...,...,...
20190,unconfirmed,Bronx,False,moderate,Shared room,2007,140,2,2,702
20191,unconfirmed,Brooklyn,False,flexible,Private room,2008,159,5,0,794
20192,unconfirmed,Manhattan,False,flexible,Entire home/apt,2013,143,11,14,715
20193,verified,Brooklyn,True,flexible,Private room,2019,22,5,1,111


In [22]:
#I created an inference funtion
def inference(df):
    df=df.drop('price',axis=1)
    train_dicts = df.to_dict(orient='records')
    X = dv.transform(train_dicts)

    y_pred = lr.predict(X)
    
    
    return y_pred

In [23]:
new=inference(val1)
new

array([155.01147234, 160.04562771, 819.99575755, ..., 714.99458527,
       110.03200415, 395.01235643])

In [26]:
r2_score(val1['price'],new)

0.9999816662514853

In [27]:
t1=preprocess("./test.csv")
t2=inference(t1)
r2_score(t1[target],t2)

0.9999817818240743

In [28]:
import mlflow

In [29]:
print(mlflow.get_tracking_uri())

file:///C:/Users/USER/Documents/Data/personal/mlops-zoomcamp/FINAL%20PROJECT/Airbnb_Open_Data.csv/mlruns


In [30]:
mlflow.list_experiments()

[<Experiment: artifact_location='./mlruns/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>]

In [31]:
lr=LinearRegression()
knn=KNeighborsRegressor()
dt=DecisionTreeRegressor()
rf=RandomForestRegressor()
gb=GradientBoostingRegressor()
svr=LinearSVR()
models=[lr,knn,dt,rf,gb,svr]

In [37]:
X_val=val1.drop('price',axis=1)
X_val= X_val.to_dict(orient='records')
X_val = dv.transform(X_val)
y_val=val1['price']

In [38]:
for model in models:
    with mlflow.start_run():

        mlflow.set_experiment('my_experiment')

        mlflow.set_tag("Developer", "Ezinwanne")

        mlflow.log_param("train-data-path", "./train.csv")
        mlflow.log_param("valid-data-path", "./val.csv")


        model.fit(X_train, y_train)

        y_pred = model.predict(X_val)
        R2 = r2_score(y_val, y_pred)
        mlflow.log_metric("R2 SCORE", R2)
        
        with open("models/preprocessor.b", "wb") as f_out:
            pickle.dump(dv, f_out)
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")
        
        with open(f'models/{model}.bin', 'wb') as f_out:
            pickle.dump((dv, model), f_out)
        mlflow.sklearn.log_model(model,artifact_path="models")
        mlflow.log_artifact(local_path=f"models/{model}.bin", artifact_path="models")

C:\Users\USER\AppData\Roaming\Python\Python38\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


In [42]:
import xgboost as xgb 

In [43]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [44]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [45]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [46]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[13:47:08] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:346.45398                                                                                          
[1]	validation-rmse:169.68663                                                                                          
[2]	validation-rmse:83.11362                                                                                           
[3]	validation-rmse:40.72263                                                                                           
[4]	validation-rmse:19.97823                                                                                           
[5]	validation-rmse:9.85996                                                                                            
[6]	validation-rmse:4.99716                                                                            

[1]	validation-rmse:553.93738                                                                                          
[2]	validation-rmse:490.23392                                                                                          
[3]	validation-rmse:433.85342                                                                                          
[4]	validation-rmse:383.95886                                                                                          
[5]	validation-rmse:339.80133                                                                                          
[6]	validation-rmse:300.72192                                                                                          
[7]	validation-rmse:266.13592                                                                                          
[8]	validation-rmse:235.52914                                                                                          
[9]	validation-rmse:208.44192           

[137]	validation-rmse:1.34472                                                                                          
[138]	validation-rmse:1.34498                                                                                          
[139]	validation-rmse:1.34511                                                                                          
[140]	validation-rmse:1.34516                                                                                          
[141]	validation-rmse:1.34526                                                                                          
[142]	validation-rmse:1.34535                                                                                          
[143]	validation-rmse:1.34551                                                                                          
[144]	validation-rmse:1.34560                                                                                          
[145]	validation-rmse:1.34559           

[96]	validation-rmse:1.34917                                                                                           
[97]	validation-rmse:1.34954                                                                                           
[98]	validation-rmse:1.34962                                                                                           
[99]	validation-rmse:1.34969                                                                                           
[100]	validation-rmse:1.34993                                                                                          
[101]	validation-rmse:1.34980                                                                                          
[102]	validation-rmse:1.34980                                                                                          
[103]	validation-rmse:1.34984                                                                                          
[104]	validation-rmse:1.34989           

[22]	validation-rmse:1.40374                                                                                           
[23]	validation-rmse:1.40366                                                                                           
[24]	validation-rmse:1.40355                                                                                           
[25]	validation-rmse:1.40351                                                                                           
[26]	validation-rmse:1.40335                                                                                           
[27]	validation-rmse:1.40338                                                                                           
[28]	validation-rmse:1.40344                                                                                           
[29]	validation-rmse:1.40346                                                                                           
[30]	validation-rmse:1.40340            

...

[70]	validation-rmse:3.32010                                                                                           
[71]	validation-rmse:3.11724                                                                                           
[72]	validation-rmse:2.93201                                                                                           
[73]	validation-rmse:2.76306                                                                                           
[74]	validation-rmse:2.60946                                                                                           
[75]	validation-rmse:2.46968                                                                                           
[76]	validation-rmse:2.34304                                                                                           
[77]	validation-rmse:2.22839                                                                                           
[78]	validation-rmse:2.12514            

[206]	validation-rmse:1.33706                                                                                          
[207]	validation-rmse:1.33706                                                                                          
[208]	validation-rmse:1.33706                                                                                          
[209]	validation-rmse:1.33706                                                                                          
[210]	validation-rmse:1.33706                                                                                          
[211]	validation-rmse:1.33706                                                                                          
[212]	validation-rmse:1.33706                                                                                          
[213]	validation-rmse:1.33706                                                                                          
[214]	validation-rmse:1.33706           

[121]	validation-rmse:1.57624                                                                                          
[122]	validation-rmse:1.55241                                                                                          
[123]	validation-rmse:1.53071                                                                                          
[124]	validation-rmse:1.51103                                                                                          
[125]	validation-rmse:1.49315                                                                                          
[126]	validation-rmse:1.47683                                                                                          
[127]	validation-rmse:1.46219                                                                                          
[128]	validation-rmse:1.44884                                                                                          
[129]	validation-rmse:1.43685           

[257]	validation-rmse:1.32515                                                                                          
[258]	validation-rmse:1.32515                                                                                          
[259]	validation-rmse:1.32515                                                                                          
[260]	validation-rmse:1.32515                                                                                          
[261]	validation-rmse:1.32515                                                                                          
[262]	validation-rmse:1.32515                                                                                          
[263]	validation-rmse:1.32515                                                                                          
[264]	validation-rmse:1.32516                                                                                          
[265]	validation-rmse:1.32516           

KeyboardInterrupt: 